# Iterative methods for solving linear systems




Recall the prototypal PDE problem introduced in the Lecture 08:
$$
-u_{xx}(x) = f(x)\quad\mathrm{ in }\ \Omega = (0, 1)
$$
$$
u(x) = 0, \quad\mathrm{ on }\ \partial\Omega = \{0, 1\}
$$

For the numerical discretization of the problem, we consider a **Finite Difference (FD) Approximation**. Let $n$ be an integer, a consider a uniform subdivision of the interval $(0,1)$ using $n$ equispaced points, denoted by $\{x_i\}_{i=0}^n$ . Moreover, let $u_i$ be the FD approximation of $u(x_i)$, and similarly $f_i \approx f(x_i)$.

The linear system that we need to solve is
$$
u_i = 0 \qquad\qquad\qquad\qquad i=0,
$$
$$
\frac{-u_{i-1} + 2u_i - u_{i+1}}{h^2} = f_i \qquad\qquad\qquad i=1, \ldots, n-1,\qquad\qquad\qquad(P)
$$
$$
u_i = 0 \qquad\qquad\qquad\qquad i=n.
$$

In [1]:
%matplotlib inline
import numpy as np
from matplotlib.pyplot import *

n = 33
h = 1./(n-1)

x=np.linspace(0,1,n)

a = -np.ones((n-1,)) # Offdiagonal entries
b = 2*np.ones((n,)) # Diagonal entries
A = (np.diag(a, -1) + np.diag(b, 0) + np.diag(a, +1))
A /= h**2
f = x*(1.-x)

# Change first row of the matrix A
A[0,:] = 0
A[:,0] = 0
A[0,0] = 1
f[0] = 0

# Change last row of the matrix A
A[-1,:] = 0
A[:,-1] = 0
A[-1,-1] = 1
f[-1] = 0

# Solution by direct method
u = np.linalg.solve(A, f)

## Jacobi

$$ 
x_i^{k+1} = \frac{1}{A_{ii}} \times \left(b_i - \sum_{j\neq i} a_{ij}x_j^k\right)
$$



In [5]:
def jacobi(A, b, nmax=10000, eps=1e-10):
    err=1.0
    n=0
    sol=np.zeros((len(b)))
    sol_new=np.zeros((len(b)))
    while err>eps and n<nmax : 
        for i in range(len(b)):
            sol_new[i]=(1/A[i,i])*(b[i]-np.dot(A[i,0:i],sol[0:i])-np.dot(A[i,i+1:len(b)],sol[i+1:len(b)]))
        err = max(np.abs( sol_new - sol ) )
        n +=1
        sol[:]=sol_new[:]
    print("final err=",err)
    print("final iter=",n)
    return sol_new

sol_jacobi = jacobi(A, f)
print(np.linalg.norm(sol_jacobi - u)/np.linalg.norm(u))

final err= 9.953820406805036e-11
final iter= 2912
7.862795872145407e-07


## Gauss-Seidel

$$ 
x_i^{k+1} = \frac{1}{A_{ii}} \times \left(b_i - \sum_{j=0}^{i-1} a_{ij}x_j^{k+1} - \sum_{j=i+1}^{N} a_{ij}x_j^k\right)
$$

In [4]:
def gauss_seidel(A,b,nmax=10000, eps=1e-10):
    err=1.0
    n=0
    sol=np.zeros((len(b)))
    sol_new=np.zeros((len(b)))
    while err>eps and n<nmax : 
        for i in range(len(b)):
            sol_new[i]=(1/A[i,i])*(b[i]-np.dot(A[i,0:i],sol_new[0:i])-np.dot(A[i,i+1:len(b)],sol[i+1:len(b)]))
        err = max(np.abs( sol_new - sol ) )
        n +=1
        sol[:]=sol_new[:]
    print("final err=",err)
    print("final iter=",n)
    return sol_new

sol_gauss_seidel = gauss_seidel(A, f)
print(np.linalg.norm(sol_gauss_seidel - u)/np.linalg.norm(u))

final err= 9.986736784761696e-11
final iter= 1528
3.9379697006483356e-07


   ## Gradient method
   $$
   {\bf r}^k = {\bf b} - A {\bf x}^k
   $$
   
   $$
   \alpha^k = \frac{{\bf r}^{k^{T}} {\bf r}^k}{{\bf r}^{k^{T}} A{\bf r}^k}
   $$
   
   $$
   {\bf x}^{k+1} = {\bf x}^k + \alpha^k {\bf r}^k
   $$
   
   ### Preconditioned gradient method
   $$
   P{\bf z}^k =  {\bf r}^k
   $$
   
   $$
   \alpha^k = \frac{{\bf z}^{k^{T}} {\bf r}^k}{{\bf z}^{k^{T}} A{\bf z}^k}
   $$
 
   $$
   {\bf x}^{k+1} = {\bf x}^k + \alpha^k {\bf z}^k
   $$ 
   
   $$
   {\bf r}^{k+1} = {\bf r}^k  - \alpha^k A{\bf z}^k
   $$

In [8]:
def gradient(A, b, P, nmax=8000, eps=1e-10):
    err=1.0
    n=0
    sol=np.zeros((len(b)))
    r=b-np.dot(A,sol)
    while err>eps and n<nmax : 
        z=np.linalg.solve(P,r)
        alpha=np.dot(z,r)/np.dot(z,np.dot(A,z))
        sol=sol + alpha*z
        r=r-alpha*np.dot(A,z)
        err = np.linalg.norm(r,2)
        n +=1
    print("final err=",err)
    print("final iter=",n)
    return sol
    
sol_gradient = gradient(A, f, np.identity(len(A)))
print(np.linalg.norm(sol_gradient - u)/np.linalg.norm(u))
print()
sol_preconditioned_gradient = gradient(A, f, A)
print(np.linalg.norm(sol_preconditioned_gradient - u)/np.linalg.norm(u))

final err= 9.947539341894685e-11
final iter= 3909
7.0953071470574e-11

final err= 1.783185031560722e-14
final iter= 1
6.888262765579997e-15


## Conjugate gradient
   
   $$
   \alpha^k = \frac{{\bf p}^{k^{T}} {\bf r}^k}{{\bf p}^{k^{T}} A{\bf p}^k}
   $$
   
      
   $$
   {\bf x}^{k+1} = {\bf x}^k + \alpha^k {\bf p}^k
   $$
   
   $$
   {\bf r}^{k+1} = {\bf r}^k - \alpha^kA {\bf p}^k
   $$

   $$
   \beta^k = \frac{(A{\bf p}^{k})^{T}{\bf r}^{k+1}}{(A{\bf p}^{k})^{T}  {\bf p}^k}
   $$
   
   $$
   {\bf p}^{k+1} = {\bf r}^{k+1} - \beta^k{\bf p}^k
   $$

   
   ### Preconditioned conjugate gradient
   
   
   $$
   \alpha^k = \frac{{\bf p}^{k^{T}} {\bf r}^k}{(A{\bf p}^{k})^{T}{\bf p}^k}
   $$
   
      
   $$
   {\bf x}^{k+1} = {\bf x}^k + \alpha^k {\bf p}^k
   $$
   
   $$
   {\bf r}^{k+1} = {\bf r}^k - \alpha^kA {\bf p}^k
   $$

$$
P{\bf z}^{k+1} = {\bf r}^{k+1}
$$

   $$
   \beta^k = \frac{(A{\bf p}^{k})^{T}{\bf z}^{k+1}}{{\bf p}^{k^T}A  {\bf p}^k}
   $$
   
   $$
   {\bf p}^{k+1} = {\bf z}^{k+1} - \beta^k{\bf p}^k
   $$


In [20]:
def conjugate_gradient(A, b, P, nmax=len(A), eps=1e-10):
    err=1.0
    n=0
    sol=np.zeros((len(b)))
    r=b-np.dot(A,sol)
    z=np.linalg.solve(P,r)
    p=np.zeros((len(b)))
    p[:]=z[:]
    while err>eps and n<nmax : 
        alpha=np.dot(p,r)/np.dot(np.dot(A,p),p)
        sol=sol + alpha*p
        r=r-alpha*np.dot(A,p)
        z=np.linalg.solve(P,r)
        beta=np.dot(np.dot(A,p),z)/np.dot(np.dot(p,A),p)
        p=z-beta*p
        err = np.linalg.norm(r,2)
        n +=1
    print("final err=",err)
    print("final iter=",n)
    return sol

sol_conjugate_gradient = conjugate_gradient(A, f, np.identity(len(A)))
print(np.linalg.norm(sol_conjugate_gradient - u)/np.linalg.norm(u))

final err= 8.053291121098274e-17
final iter= 16
2.9030318378385336e-15
